In [1]:
import pandas as pd

df = pd.read_csv("./data/Movie250k.csv")


In [2]:
df = df.drop(['Unnamed: 0','overview'], axis=1)

In [3]:
df.isnull().sum()

id                       0
title                    0
genres               26184
original_language        0
popularity               0
release_date          3121
budget                   0
revenue                  0
runtime               3221
status                   0
vote_average             0
vote_count               0
dtype: int64

In [4]:
#checking most common release year
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_date'].dt.year.value_counts()

release_date
2019.0    13389
2020.0    12195
2021.0    12072
2018.0    11982
2017.0    11234
          ...  
1883.0        1
1885.0        1
1881.0        1
1889.0        1
1882.0        1
Name: count, Length: 148, dtype: int64

In [5]:
#fill missing release dates with column mode
df['release_date'].dt.year.value_counts()
df['release_date'] = df['release_date'].fillna('2019-01-01')
df['release_date'] = pd.to_datetime(df['release_date'])


In [6]:
#changing missing genre to unknown
df['genres'] = df['genres'].fillna('Unknown')

In [7]:
#get dummy variables for movie genre
df.genres
out = []
for a in df.genres:
    out.append(a.split('-'))
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
binarizer = MultiLabelBinarizer()
binarized_genres = binarizer.fit_transform(np.array(out,dtype="object"))
genre = binarizer.classes_
df[genre] = binarized_genres
df.drop(columns='genres', inplace=True)

In [8]:
df[(df['budget']>1) & (df['revenue']>1)]

,id,title,original_language,popularity,release_date,budget,revenue,runtime,status,vote_average,...,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,Unknown,War,Western
0,615656,Meg 2: The Trench,en,8763.998,2023-08-02,129000000.0,352056482.0,116.0,Released,7.079,...,1,0,0,0,1,0,0,0,0,0
1,758323,The Pope's Exorcist,en,5953.227,2023-04-05,18000000.0,65675816.0,103.0,Released,7.433,...,1,0,1,0,0,0,1,0,0,0
2,667538,Transformers: Rise of the Beasts,en,5409.104,2023-06-06,200000000.0,407045464.0,127.0,Released,7.340,...,0,0,0,0,1,0,0,0,0,0
3,640146,Ant-Man and the Wasp: Quantumania,en,4425.387,2023-02-15,200000000.0,475766228.0,125.0,Released,6.507,...,0,0,0,0,1,0,0,0,0,0
4,677179,Creed III,en,3994.342,2023-03-01,75000000.0,269000000.0,116.0,Released,7.262,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248688,295448,His People,en,0.878,1925-11-01,93000.0,3000000.0,93.0,Released,0.000,...,0,0,0,0,0,0,0,1,0,0
248695,541732,Privacy,fa,0.878,2017-08-11,10000.0,100000.0,76.0,Released,4.000,...,0,0,0,0,0,0,0,0,0,0
249609,335223,Savage High,en,0.877,2015-10-30,3.0,2.0,NaN,Released,9.000,...,1,0,1,0,0,0,1,0,0,0
249718,618187,After Hours,en,0.876,2015-04-25,15.0,175.0,3.0,Released,6.000,...,1,0,0,0,0,0,0,0,0,0


In [9]:
#replacing runtime missing values and 0 values with column mean
df['runtime'] = df['runtime'].fillna(df[df['runtime']>0]['runtime'].mean())
df['runtime'] = df['runtime'].replace(0.0,df[df['runtime']>0]['runtime'].mean())
df['runtime'].describe()

count    250000.000000
mean         86.021490
std         145.260658
min           1.000000
25%          76.000000
50%          87.000000
75%         100.000000
max       51420.000000
Name: runtime, dtype: float64

In [10]:
#Impute vote average with column mode if there is a significant number of vote count and a vote average of 0
df.loc[((df['vote_count']>10) & (df['vote_average']==0)),'vote_average'] = df[df['vote_count']>0]['vote_average'].mode()[0]

In [11]:
df.isnull().sum()

id                   0
title                0
original_language    0
popularity           0
release_date         0
budget               0
revenue              0
runtime              0
status               0
vote_average         0
vote_count           0
Action               0
Adventure            0
Animation            0
Comedy               0
Crime                0
Documentary          0
Drama                0
Family               0
Fantasy              0
History              0
Horror               0
Music                0
Mystery              0
Romance              0
Science Fiction      0
TV Movie             0
Thriller             0
Unknown              0
War                  0
Western              0
dtype: int64